In [13]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [14]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', 1500)        # Let the display adjust to the window


In [15]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean_stocks_etfs.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios_stocks_etfs.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-25
DEST_DIR: ..\data

source_path: ..\data\2025-04-25_df_OHLCV_clean_stocks_etfs.parquet
dest_path: ..\data\2025-04-25_df_perf_ratios_stocks_etfs.parquet


In [16]:
import pandas as pd

# Get tickers from df_finviz_stocks_etfs.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz_stocks_etfs.parquet')
tickers = df_finviz.index.to_list()

In [17]:
print(df_finviz) 

        No.                                     Company               Index                  Sector                        Industry Country Exchange  Market Cap, M    P/E  Fwd P/E   PEG    P/S    P/B    P/C  P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %    EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta    ATR  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %        52W Range  All-Time High %  All-Time Low %    RSI  Earnings    IP

In [18]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
print(df)

                     Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
WTFC   2025-04-25  110.09  111.35  109.29  110.79     110.79   451000  110.090000  111.350000  109.290000
       2025-04-24  107.33  111.67  106.86  111.38     111.38   485000  107.330000  111.670000  106.860000
       2025-04-23  109.98  112.94  106.82  107.68     107.68   763800  109.980000  112.940000  106.820000
       2025-04-22  100.64  106.94  100.64  106.66     106.66   843100  100.640000  106.940000  100.640000
       2025-04-21  102.20  102.90  100.37  101.43     101.43   437700  102.200000  102.900000  100.370000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
AEG    2024-05-02    6.28    6.30    6.21    6.26       5.91  3340100    5.928882    5.947764    5.862796
       2024-05-01    6.16    6.32    6.15    6

In [19]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
WTFC   2025-04-25     110.79
       2025-04-24     111.38
       2025-04-23     107.68
       2025-04-22     106.66
       2025-04-21     101.43
...                      ...
AEG    2024-05-02       5.91
       2024-05-01       5.89
       2024-04-30       5.82
       2024-04-29       5.90
       2024-04-26       5.85

[384250 rows x 1 columns]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 384250 entries, ('WTFC', Timestamp('2025-04-25 00:00:00')) to ('AEG', Timestamp('2024-04-26 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  384250 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [21]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-25     209.28
       2025-04-24     208.37
       2025-04-23     204.60
       2025-04-22     199.74
       2025-04-21     193.16
...                      ...
FELG   2024-05-02      28.50
       2024-05-01      28.11
       2024-04-30      28.19
       2024-04-29      28.71
       2024-04-26      28.68

[384250 rows x 1 columns]


In [22]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [23]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
       Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A       5.153101   19.052913  2.697368  15.793771   65.559711  9.259747    3.204522     5.896718   1.619189    1.202716     1.867554   1.226958   -2.257962    -2.893040   0.672414   -3.871887    -4.635301   0.500543    -1.240867     -1.661360    0.804215    -0.846424     -1.118219    0.859558
AA      3.723058   11.141484  1.992562  10.880600   28.880362  4.638584    1.956660     2.954931   1.380855    1.063227     1.742029   1.200455   -2.404414    -3.160253   0.660904   -1.736624    -2.277000   0.738112    -1.411740     -1.848063    0.780096    -0.480904     -0.665725    0.922313
AAL    30.773939         inf       NaN  21.440824         inf       NaN    0.

In [27]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, A to ZWS
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sharpe 3d     1537 non-null   float64
 1   Sortino 3d    1537 non-null   float64
 2   Omega 3d      809 non-null    float64
 3   Sharpe 5d     1537 non-null   float64
 4   Sortino 5d    1537 non-null   float64
 5   Omega 5d      938 non-null    float64
 6   Sharpe 10d    1537 non-null   float64
 7   Sortino 10d   1537 non-null   float64
 8   Omega 10d     1533 non-null   float64
 9   Sharpe 15d    1537 non-null   float64
 10  Sortino 15d   1537 non-null   float64
 11  Omega 15d     1537 non-null   float64
 12  Sharpe 30d    1537 non-null   float64
 13  Sortino 30d   1537 non-null   float64
 14  Omega 30d     1537 non-null   float64
 15  Sharpe 60d    1537 non-null   float64
 16  Sortino 60d   1537 non-null   float64
 17  Omega 60d     1537 non-null   float64
 18  Sharpe 120d   1537 non-null   floa

In [28]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            # NaN is from Omega ratio where daily loss series are all zero
            # +Inf is from Sharpe ratio where daily negative excess return series are all zero
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            # -Inf is from Sharpe ratio where daily excess return series are the same, therefore standard deviation is zero
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['PTLC']
Column: Sortino 3d
  Inf: ['AAL', 'AAON', 'AAPL', 'ABBV', 'ABEV', 'ABNB', 'ACIW', 'ACN', 'ACWI', 'ACWX', 'ADBE', 'AEG', 'AER', 'AFRM', 'AGG', 'AGNC', 'AKAM', 'ALAB', 'ALC', 'ALNY', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'AN', 'ANET', 'ANSS', 'APA', 'APG', 'APO', 'APP', 'APTV', 'AR', 'ARCC', 'ARES', 'ARGX', 'ARKB', 'ARKK', 'ARM', 'AS', 'ASML', 'ASND', 'ASR', 'ASX', 'ATI', 'AUR', 'AVDE', 'AVDV', 'AVGO', 'AVUS', 'AWI', 'AXON', 'AZN', 'BA', 'BABA', 'BAC', 'BAH', 'BALL', 'BAM', 'BAP', 'BBEU', 'BBIN', 'BBIO', 'BBJP', 'BBUS', 'BBVA', 'BBY', 'BCH', 'BDX', 'BECN', 'BEP', 'BG', 'BIDU', 'BILS', 'BINC', 'BIRK', 'BITB', 'BIV', 'BKNG', 'BKR', 'BLDR', 'BLV', 'BMRN', 'BN', 'BND', 'BNS', 'BNT', 'BOKF', 'BOND', 'BP', 'BPMC', 'BRBR', 'BRFS', 'BRKR', 'BROS', 'BSAC', 'BSCQ', 'BSCR', 'BSV', 'BSX', 'BUFR', 'BURL', 'BWXT', 'BXSL', 'C', 'CAE', 'CAH', 'CALF'

In [29]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'AAL'

#### Use Cell below to check the inf, -inf and NaN replacement

In [30]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       30.773939
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       21.440824
Sortino 5d            inf
Omega 5d              NaN
Sharpe 10d       0.627020
Sortino 10d      0.870148
Omega 10d        1.105977
Sharpe 15d       0.959973
Sortino 15d      1.686722
Omega 15d        1.221849
Sharpe 30d      -0.564022
Sortino 30d     -0.880908
Omega 30d        0.893880
Sharpe 60d      -2.831226
Sortino 60d     -3.910800
Omega 60d        0.571980
Sharpe 120d     -0.808785
Sortino 120d    -1.219591
Omega 120d       0.848549
Sharpe 250d     -0.487410
Sortino 250d    -0.718294
Omega 250d       0.907541
Name: AAL, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d          30.773939
Sortino 3d       3802.268850
Omega 3d          339.733032
Sharpe 5d          21.440824
Sortino 5d      15666.484213
Omega 5d         1974.791483
Sharpe 10d          0.627020
Sortino 10d         0.870148
Omega 10d           1.105977
Sharpe 15d          0.959973
Sortino 15d         1.686722
Omega 15d           1.221849
Sharpe 30d         -0.564022
Sortino 30d        -0.880908
Omega 30d           0.893880
Sharpe 60d         -2.831226
Sortino 60d        -3.910800
Omega 60d           0.571980
Sharpe 120d        -0.808785
Sortino 120d       -1.219591
Omega 120d          0.848549
Sharpe 250d        -0.487410
Sortino 250d       -0.718294
Omega 250d          0.907541
Name: AAL, dtype: float64

In [31]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-25_df_perf_ratios_stocks_etfs.parquet


In [32]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'AMZN', 'TSLA', 'META', 'NFLX', 'V', 'JPM', 'UNH']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====